# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import gmaps

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("../output_data/weather.csv", dtype = "object", 
                         encoding="utf-8")
weather_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.5,7.16
1,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.7,78.8,11.41
2,2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.2,2.37
3,3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.6,4.47
4,4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...,...
543,543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.6,8.05
544,544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.2,16.11
545,545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.0,8.05
546,546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Storing "Lat" and "Lng" as location variable
location = weather_df[["Lat", "Lng"]].astype(float)
#Storing "Humidity" as weight
weight = weather_df["Humidity"]

#Adding Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weight,
                                dissipating=False, max_intensity= 100,
                                point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Creating new DataFrame using "Max Temp", "Wind Speed", and "Cloudiness"
ideal_temp_df = weather_df[["Max Temp", "Wind Speed", ]].astype(float)

# Since "Cloudiness" is int dtype, "Max Temp" & "Wind Speed" are float, need to add new column
#for "Cloudiness" because of different dtypes
ideal_temp_df["Cloudiness"] = weather_df["Cloudiness"]
ideal_temp_df

#Narrowing down by "Max Temp"
ideal_temp_df = ideal_temp_df.loc[ideal_temp_df["Max Temp"] < 80]

ideal_temp_df = ideal_temp_df.loc[ideal_temp_df["Max Temp"] > 70]

ideal_wind_df = ideal_temp_df

In [14]:
# Narrowing down by "Wind Speed" ("Wind Speed" > 10 mph)
ideal_wind_df = ideal_wind_df.loc[ideal_wind_df["Wind Speed"] < 10]
ideal_wind_df

,Max Temp,Wind Speed,Cloudiness
7,72.06,5.44,0
12,71.60,2.24,100
25,78.80,6.93,20
44,79.00,5.99,26
59,74.22,2.62,54
70,73.68,5.32,0
87,74.76,4.52,2
88,76.56,5.50,0
91,75.66,7.29,62
100,77.28,2.01,95


In [19]:
# Narrowing down by "Cloudiness" ("Cloudiness" == 0)
ideal_weather_df = ideal_wind_df.loc[ideal_wind_df["Cloudiness"] == 0]
ideal_weather_df.head()

,Max Temp,Wind Speed,Cloudiness


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map